#E-mail Spam Filtering based on Naiive Bayes

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, precision_score

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[+-_/(){}!^?<>"''\[\]\|,;:.]')
# data cleaning
def clean_text(text):
  # change to lower-case
  #text = str(text).lower()
  text = text.replace('\n', ' ')
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = list(set(text.split(' ')))
  return text

def clean_text1(text):
  # change to lower-case
  #text = str(text).lower()
  text = text.replace('\n', ' ')
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = list(text.split(' '))
  return text

## 1. feature selection using the information gain (IG) metric

In [ ]:
corpus = ''
legit = []
spam = []

# read the train data and split them into spam and legit
for i in range(1, 10):
  root = '/content/drive/My Drive/Colab Notebooks/data/lemm_stop/part' + str(i)
  file_names = os.listdir(root)

  for file in file_names:
    f = open(root + '/' + file, 'r')
    text = f.read()
    corpus = corpus + text
    text = clean_text(text)
    if file[0] != 's':
      legit.append(text)
    else:
      spam.append(text)
    f.close()

print(len(legit))
print(len(spam))
##print(corpus)

2170
432


In [ ]:
corpus = clean_text(corpus)
print('Total number of words in corpus:', len(corpus))

n_spam = len(spam)       # total number of spam email
n_legit = len(legit)      # total number of legit email
n_email = n_spam + n_legit  # total number of email

# compute inherent uncertainty
p = n_legit/(n_spam + n_legit)
HC = -p*np.log(p) - (1-p)*np.log(1-p)
print('HC:', HC)

Total number of words in corpus: 46122
HC: 0.4495288323617157


In [ ]:
# compute IG for each term in the corpus
IG = {}             # store the IG value for all terms
for term in corpus:
  term = term.replace("'", "")
  spam_count = 0    # number of spam email with term
  legit_count = 0    # number of legit email with term
  # compute the number of legit and spam with each term in corpus
  for i in range(len(spam)):
    if term in spam[i]:
      spam_count += 1
  for j in range(len(legit)):
    if term in legit[j]:
      legit_count += 1

  n_1 = spam_count + legit_count    # number of emails with term
  n_0 = n_email - n_1         # number of emails without term
  
  # use smoothing for terms never occur
  if n_1 == 0:
    spam_count += 1
    legit_count += 1
    n_1 = spam_count + legit_count
    n_0 = n_email - n_1

  if n_0 == 0:   # all the email contain this term
    continue

  H_s0 = ((n_spam-spam_count)/n_email) * (np.log((n_spam-spam_count)/n_0))  # spam without term
  H_s1 = (spam_count/n_email) * (np.log(spam_count/n_1))   # spam with term
  H_l0 = ((n_legit-legit_count)/n_email) * (np.log((n_legit-legit_count)/n_0))  # legit without term
  H_l1 = (legit_count/n_email) * (np.log(legit_count/n_1))   # legit with term

  H_term = (-1) * (H_s0 + H_s1 + H_l0 + H_l1)    # entropy(uncertainty) after this term

  ig = HC - H_term             # information gain
  IG[term] = [spam_count, legit_count, ig]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
# sort IG value
df = pd.DataFrame(data=IG.values(), index=IG.keys(), columns=['#spam with term', '#legit with term', 'IG'])
df = df[df['IG'] != -np.inf]
df.sort_values(by=['IG'], inplace=True, ascending=False)
print('Total number of spam email:', n_spam)
print('Total number of legit email:', n_legit)
df

Total number of spam email: 432
Total number of legit email: 2170


,#spam with term,#legit with term,IG
language,7,1453,0.142768
remove,186,23,0.117098
free,253,138,0.113103
university,17,1276,0.100511
money,169,58,0.082261
...,...,...,...
tonus,0,3,NaN
shandong,0,1,NaN
circumscriptional,0,1,NaN
seretha,0,1,NaN


In [ ]:
df.to_csv('terms.csv')

#Run code from here
##2. Three classifiers

In [ ]:
n_spam = 432           # number of spam emails
n_legit = 2170            # number of legit emails
n_email = n_spam + n_legit      # total number of emails

In [ ]:
# read the IG data
df = pd.read_csv('terms.csv')
#df = pd.read_excel('terms.xlsx')
df.index = df['Unnamed: 0']
df.drop(['Unnamed: 0'], axis=1, inplace=True)

# get topN terms, N = 10, 100, 1000
df_1 = df.iloc[0:10]
df_2 = df.iloc[0:100]
df_3 = df.iloc[0:1000]
#df_4 = df.iloc[0:5000]
df_1

,#spam with term,#legit with term,IG
Unnamed: 0,,,
language,7,1453,0.142768
remove,186,23,0.117098
free,253,138,0.113103
university,17,1276,0.100511
money,169,58,0.082261
click,117,13,0.070879
market,132,41,0.064122
business,143,67,0.060185
today,144,79,0.056415


## Bernoulli NB classifier with binary features

In [ ]:
# build Bernoulli NB classifier
def Bern_NB(text, df, n_spam=n_spam, n_legit=n_legit, n_email=n_email):
  text = clean_text(text)
  Pr_s = n_spam/n_email
  Pr_l = n_legit/n_email
  Pr_is = 1
  Pr_il = 1
  
  for i in range(len(df)):
    spam_count = df.iloc[i][0]      # get the number of spam with term i 
    legit_count = df.iloc[i][1]      # get the number of legit with term i 

    if df.index[i] in text:         # this email has term i 
      Pr_is = ((spam_count+1)/(n_spam+2))*Pr_is
      Pr_il = ((legit_count+1)/(n_legit+2))*Pr_il
    else:                     # this email without term i
      Pr_is = (1-((spam_count+1)/(n_spam+2)))*Pr_is
      Pr_il = (1-((legit_count+1)/(n_legit+2)))*Pr_il

  Pr_si = (Pr_is * Pr_s)#/Pr_i       # Pr of spam
  Pr_li = (Pr_il * Pr_l)#/Pr_i       # Pr of legit

  return Pr_si/Pr_li

##Multinomial NB with binary features

In [ ]:
'''
# compute the number of occurrences for each term in spam and ham
legit = []
spam = []

# read the train data and split them into spam and legit
for i in range(1, 10):
  root = '/content/drive/My Drive/Colab Notebooks/data/lemm_stop/part' + str(i)
  file_names = os.listdir(root)

  for file in file_names:
    f = open(root + '/' + file, 'r')
    text = f.read()
    #corpus = corpus + text
    text = clean_text1(text)
    if file[0] != 's':
      legit.append(text)
    else:
      spam.append(text)
    f.close()

print(len(spam))
print(len(legit))
'''

432
2170


In [ ]:
'''
occur = {}
for term in df_3.index:
  #term = term.replace("'", "")
  spam_occ = 0    # number of spam email with term
  legit_occ = 0    # number of legit email with term
  # compute the number of legit and spam with each term in corpus
  for i in range(len(spam)):
    for j in range(len(spam[i])):
      if term == spam[i][j]:
        spam_occ += 1
  for i in range(len(legit)):
    for j in range(len(legit[i])):
      if term == legit[i][j]:
        legit_occ += 1
  occur[term] = [spam_occ, legit_occ]
'''

In [ ]:
'''
df0 = pd.DataFrame(data=occur.values(), index=occur.keys(), columns=['#term in spam', '#term in legit'])
df0.to_csv('term1000_occur.csv')
'''

In [ ]:
# read the 1000 terms occurrence data 
df0 = pd.read_csv('term1000_occur.csv')
df0.index = df0['Unnamed: 0']
df0.drop(['Unnamed: 0'], axis=1, inplace=True)
'''
nterm_s = 0
nterm_l = 0
for i in range(len(spam)):
  nterm_s += len(spam[i])
for i in range(len(legit)):
  nterm_l += len(legit[i])
'''
nterm_s = 454430
nterm_l = 1488668
print('Total number of terms in spam:', nterm_s)
print('Total number of terms in legit:', nterm_l)

# get topN terms, N = 10, 100, 1000
df1 = df0.iloc[0:10]
df2 = df0.iloc[0:100]
df3 = df0
df1

Total number of terms in spam: 454430
Total number of terms in legit: 1488668


,#term in spam,#term in legit
Unnamed: 0,,
language,19,7593
remove,415,30
free,905,196
university,39,5252
money,903,75
click,232,22
market,454,58
business,734,90
today,243,104


In [ ]:
def Multinomial_B(text, df, n_spam=n_spam, n_legit=n_legit, n_email=n_email, nterm_s=nterm_s, nterm_l=nterm_l):
  text = clean_text(text)     # unique terms
  Pr_s = n_spam/n_email
  Pr_l = n_legit/n_email
  Pr_il = 1
  Pr_is = 1

  for i in range(len(df)):
    if df.index[i] in text:
      Pr_is = ((df.iloc[i][0]+1)/(nterm_s+2))*Pr_is
      Pr_il = ((df.iloc[i][1]+1)/(nterm_l+2))*Pr_il

  Pr_si = Pr_is * Pr_s     # Pr of spam
  Pr_li = Pr_il * Pr_l     # Pr of legit

  return Pr_si/Pr_li

##Multinomial NB with TF



In [ ]:
def Multinomial_TF(text, df, n_spam=n_spam, n_legit=n_legit, n_email=n_email, nterm_s=nterm_s, nterm_l=nterm_l):
  text = clean_text1(text)
  Pr_s = n_spam/n_email
  Pr_l = n_legit/n_email
  Pr_il = 1
  Pr_is = 1

  for i in range(len(df)):
    count = 0
    for word in text:
      if df.index[i] == word:
        count += 1
    Pr_ist = ((df.iloc[i][0]+1)/(nterm_s+2))**count
    Pr_is = Pr_ist * Pr_is
    Pr_ilt = ((df.iloc[i][1]+1)/(nterm_l+2))**count
    Pr_il = Pr_ilt * Pr_il

  Pr_si = Pr_is * Pr_s     # Pr of spam
  Pr_li = Pr_il * Pr_l     # Pr of legit

  return Pr_si/Pr_li

In [ ]:
# test
result = []
result_true = []

# read the test data and split them into spam and legit
root = '/content/drive/My Drive/Colab Notebooks/data/lemm_stop/part10'
file_names = os.listdir(root)
#file_names = file_names[150:]

for file in file_names:
  f = open(root + '/' + file, 'r')
  text = f.read()
  if file[0] != 's':
    result_true.append(0)
  else:
    result_true.append(1)
  f.close()

  # prediction
  #pred = Bern_NB(text, df_3)
  #pred = Multinomial_B(text, df1)
  pred = Multinomial_TF(text, df3)

  if pred > 1:
    result.append(1)
  else:
    result.append(0)

print('Precision:', precision_score(result_true, result))
print('Recall:', recall_score(result_true, result))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in double_scalars


Precision: 0.9666666666666667
Recall: 0.5918367346938775


**REPORT**

**N = 10:**

Bernoulli_Binary: Precision: 0.89;  Recall: 0.67

Multinomial_Binary: Precision: 0.76;  Recall: 0.92

Multinomial_TF: Precision: 0.79;  Recall: 0.92

**N = 100:**

Bernoulli_Binary: Precision: 1.00;  Recall: 0.65

Multinomial_Binary: Precision: 0.76;  Recall: 0.96

Multinomial_TF: Precision: 0.80;  Recall: 0.90

**N = 1000:**

Bernoulli_Binary: Precision: 1.00;  Recall: 0.55

Multinomial_Binary: Precision: 0.97;  Recall: 0.69

Multinomial_TF: Precision: 0.94;  Recall: 0.59

* The algorithm based on Binary features run faster than that based on TF.

In [ ]:
print([result])
print([result_true])

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,